In [1]:
"""


Plott DPR VCUT etc...


"""

"""
Analyse of GPM DPR BB

"""




'\nAnalyse of GPM DPR BB\n\n'

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wradlib
import wradlib as wrl
from osgeo import osr
import h5py
import glob
from pcc import get_my_cmap
from pcc import get_miub_cmap
from pcc import boxpol_pos
bonn_pos = boxpol_pos()
bx, by = bonn_pos['gkx_ppi'], bonn_pos['gky_ppi']
bonnlat, bonnlon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']
blat, blon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']


from pcc import plot_borders
from pcc import plot_radar

#% matplotlib inline

print ("  ")

In [8]:
zt = '20141007'; pfadnr = 0
scan  = 'NS'#NS' #or MS

def read_dpr(zt, scan, pfadnr):

    pfad = ('/automount/ags/velibor/gpmdata/dpr/2A.GPM.DPR.V6-20160118.'+zt+'*.HDF5')
    dpr_pfad = sorted(glob.glob(pfad))[pfadnr]

    print dpr_pfad

    # Einlesen
    dpr = h5py.File(dpr_pfad, 'r')
    # Position
    dpr_lat=np.array(dpr[scan]['Latitude'])
    dpr_lon=np.array(dpr[scan]['Longitude'])

    # Refelctivity Volumen
    dpr_pp=np.array(dpr[scan]['SLV']['zFactorCorrected'])
    dpr_pp[dpr_pp<0]= np.nan
    print dpr_pp.shape

    #Projection
    proj_stereo = wrl.georef.create_osr("dwd-radolan")
    proj_wgs = osr.SpatialReference()
    proj_wgs.ImportFromEPSG(4326)

    from pcc import boxpol_pos
    bonn_pos = boxpol_pos()
    bx, by = bonn_pos['gkx_ppi'], bonn_pos['gky_ppi']
    bonnlat, bonnlon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']
    blat, blon = bonn_pos['lat_ppi'], bonn_pos['lon_ppi']

    dpr_lon, dpr_lat = wradlib.georef.reproject(dpr_lon, dpr_lat, projection_target=proj_stereo , projection_source=proj_wgs)
    bonnlon, bonnlat = wradlib.georef.reproject(bonnlon, bonnlat, projection_target=proj_stereo , projection_source=proj_wgs)

    # BoXPol region
    lon0, lat0, radius = bonnlon, bonnlat, 100
    r = np.sqrt((dpr_lat - lat0)**2 + (dpr_lon - lon0)**2)
    position = r < radius

    
    dpr_pp[np.where(r > radius)]=np.nan


    # Height of dpr
    h = np.arange(150,4800,150)
    if scan=='HS':
        hdpr = 1000 * (np.arange(88,0,-1)*0.250)

    else:
        hdpr = 1000 * (np.arange(176,0,-1)*0.125)

    
    return dpr_lat, dpr_lon, hdpr, dpr_pp

In [9]:
lat, lon, alt,  dpr_z = read_dpr(zt, scan, pfadnr)

/automount/ags/velibor/gpmdata/dpr/2A.GPM.DPR.V6-20160118.20141007-S015721-E032951.003445.V04A.HDF5
(7930, 49, 176)


In [10]:
print (lat.shape, lon.shape, dpr_z.shape, alt.shape)

((7930, 49), (7930, 49), (7930, 49, 176), (176,))


In [62]:
cut_entlang = 3269
cut_quer = 23
ff = 15
cc = 0.5
fig = plt.figure(figsize=(12,12))
ax1 = fig.add_subplot(221, aspect='equal')
plt.pcolormesh(lon,lat,np.ma.masked_invalid(dpr_z[:,:,160]),vmin=0, vmax=50, cmap=get_miub_cmap())
plt.plot(lon[3249:3270,cut_quer],lat[3249:3270,cut_quer], color='red')
plt.plot(lon[cut_entlang,23:43],lat[cut_entlang,23:43], color='blue', lw=2)
plot_borders(ax1)
plot_radar(bonnlon, bonnlat, ax1, reproject=True, cband=False,col='black')

plt.colorbar()
plt.xlim(-350,-100)
plt.ylim(-4350, -4100)#

plt.subplot(2,2,2)
plt.title("CUT", color='red')
plt.pcolormesh(lon[3249:3270,cut_quer],alt,np.ma.masked_invalid(dpr_z[3249:3270,cut_quer,:].transpose()),
              vmin=0, vmax=50, cmap=get_miub_cmap())
plt.colorbar()
plt.ylim(0,22000)
plt.xlim(-350,-100)

plt.subplot(2,1
            ,2)
plt.pcolormesh(lon[cut_entlang,23:43],alt,np.ma.masked_invalid(dpr_z[cut_entlang,23:43,:].transpose()),
               vmin=0, vmax=50, cmap=get_miub_cmap())
plt.colorbar()
plt.title('cut', color='blue')
plt.ylim(0,7000)
#plt.xlim(-350,-100)
plt.grid()


plt.show()


pcc.py:372: DeprecatedWarning: open_shape is deprecated as of 0.11.1 and will be removed in 1.0.0. Use :func:`open_vector` instead.
  dataset, inLayer = wradlib.io.open_shape(filename)
pcc.py:475: DeprecatedWarning: polar2polyvert is deprecated as of 0.11.3 and will be removed in 1.0.0. Use :func:`spherical_to_polyvert` instead.
  (x_loc, y_loc))
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/wradlib/georef/polar.py:673: DeprecatedWarning: polar2lonlat is deprecated as of 0.11.3 and will be removed in 1.0.0.
  lon, lat = polar2lonlat(r, az, sitecoords)
/home/velibor/anaconda2/envs/wradlib/lib/python2.7/site-packages/wradlib/georef/polar.py:140: DeprecatedWarning: hor2aeq is deprecated as of 0.11.3 and will be removed in 1.0.0.
  delta, tau = hor2aeq(a, h, phi)


In [48]:
plt.figure(figsize=(12,6))
plt.pcolormesh(lon[cut_entlang,23:43],alt,np.ma.masked_invalid(dpr_z[cut_entlang,23:43,:].transpose()),
               vmin=0, vmax=50, cmap=get_miub_cmap())
plt.colorbar()
plt.title('cut', color='blue')
plt.ylim(0,7000)
#plt.xlim(-350,-100)
plt.grid()
plt.show()
